In [1]:
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [7]:
df = pd.read_csv(r"C:\Users\Lenovo\Project\socml\data\eda\combined.csv")
df.columns = df.columns.str.strip()
print(df.columns.tolist())

['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count

In [8]:
with open(r"C:\Users\Lenovo\Project\socml\models\top_features.txt", "r", encoding="utf-8") as f:
    top_features = f.read().strip().split(",")

# Убираем лишние пробелы у названий
top_features = [col.strip() for col in top_features]

# Выбираем данные
X = df[top_features]
y = df["Label"]  # тут название колонки с целевой переменной

print(f"Используем {len(top_features)} фич: {top_features}")

Используем 10 фич: ['Packet Length Std', 'Packet Length Variance', 'Avg Bwd Segment Size', 'Max Packet Length', 'Bwd Packet Length Max', 'Bwd Packet Length Std', 'Average Packet Size', 'Total Length of Bwd Packets', 'Total Length of Fwd Packets', 'Subflow Bwd Bytes']


In [16]:
y.value_counts()

Label
BENIGN                        2096484
DoS Hulk                       172849
DDoS                           128016
PortScan                        90819
DoS GoldenEye                   10286
FTP-Patator                      5933
DoS slowloris                    5385
DoS Slowhttptest                 5228
SSH-Patator                      3219
Bot                              1953
Web Attack � Brute Force         1470
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix


In [17]:
attack_threshold = 6000
label_counts = df['Label'].value_counts()

In [18]:
major_labels = label_counts[label_counts >= attack_threshold].index.tolist()
major_labels.append('Other_atac')

In [19]:
def consolidate_labels(label):
    if label in major_labels:
        return label
    else:
        return 'Other_Attacks'

In [20]:
df['Label_Processed'] = df['Label'].apply(consolidate_labels)

print(df['Label_Processed'].value_counts())

Label_Processed
BENIGN           2096484
DoS Hulk          172849
DDoS              128016
PortScan           90819
Other_Attacks      23908
DoS GoldenEye      10286
Name: count, dtype: int64


In [23]:
le = LabelEncoder()
df['Label_Encoded'] = le.fit_transform(df['Label_Processed'])

print("Классы и их коды:")
for i, class_ in enumerate(le.classes_):
    print(f"{class_}: {i}")

Классы и их коды:
BENIGN: 0
DDoS: 1
DoS GoldenEye: 2
DoS Hulk: 3
Other_Attacks: 4
PortScan: 5


In [24]:
y = df['Label_Encoded']

In [25]:
print("Пример признаков:")
print(X.head())

Пример признаков:
   Packet Length Std  Packet Length Variance  Avg Bwd Segment Size  \
0                0.0                     0.0                   0.0   
1                0.0                     0.0                   6.0   
2                0.0                     0.0                   6.0   
3                0.0                     0.0                   6.0   
4                0.0                     0.0                   0.0   

   Max Packet Length  Bwd Packet Length Max  Bwd Packet Length Std  \
0                  6                      0                    0.0   
1                  6                      6                    0.0   
2                  6                      6                    0.0   
3                  6                      6                    0.0   
4                  6                      0                    0.0   

   Average Packet Size  Total Length of Bwd Packets  \
0                  9.0                            0   
1                  9.0        

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [27]:
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)

In [28]:
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.25, stratify=y_trainval, random_state=42)

In [29]:
print(f"Размеры выборок:")
print(f"Train: {X_train.shape}, Validation: {X_val.shape}, Test: {X_test.shape}")

Размеры выборок:
Train: (1513416, 10), Validation: (504473, 10), Test: (504473, 10)


In [30]:
import numpy as np

X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_val.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)

for col in X_train.columns:
    median = X_train[col].median()
    X_train[col] = X_train[col].fillna(median)
    X_val[col] = X_val[col].fillna(median)
    X_test[col] = X_test[col].fillna(median)

In [31]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [32]:
model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train_scaled, y_train)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [33]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_val_pred = model.predict(X_val_scaled)

print("Accuracy:", accuracy_score(y_val, y_val_pred))
print(classification_report(y_val, y_val_pred))


Accuracy: 0.9896149050593391
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    419297
           1       1.00      1.00      1.00     25603
           2       1.00      0.74      0.85      2057
           3       1.00      0.93      0.96     34570
           4       1.00      0.67      0.80      4782
           5       0.98      0.99      0.99     18164

    accuracy                           0.99    504473
   macro avg       0.99      0.89      0.93    504473
weighted avg       0.99      0.99      0.99    504473



In [34]:
y_test_pred = model.predict(X_test_scaled)

print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

Test Accuracy: 0.9894642527945005
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    419297
           1       1.00      1.00      1.00     25603
           2       0.99      0.75      0.85      2057
           3       1.00      0.93      0.96     34570
           4       1.00      0.65      0.79      4782
           5       0.98      0.99      0.99     18164

    accuracy                           0.99    504473
   macro avg       0.99      0.89      0.93    504473
weighted avg       0.99      0.99      0.99    504473



In [35]:
import joblib
joblib.dump(model, '../models/soc_model_10f).pkl')

# Сохранение списка фичей для Streamlit
with open('../models/feature_list_10.txt', 'w') as f:
    f.write(','.join(X.columns.tolist()))